In [2]:
!sudo apt-get install libkrb5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libkrb5-dev is already the newest version (1.17-6ubuntu4.2).
libkrb5-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [3]:
!pip install arcgis==1.8.4  torch==1.4.0  fastai==1.0.60 torchvision==0.5.0 spacy==2.1.8 fastprogress pandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.4/753.4 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but y

In [4]:
!pip install transformers==3.3.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00


In [5]:
from arcgis.learn.text import ZeroShotClassifier

In [6]:
classifier = ZeroShotClassifier()

/usr/local/lib/python3.8/dist-packages/arcgis/learn/models/_arcgis_model.py:336: UserWarning: Cuda is not available
  warnings.warn("Cuda is not available")


Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [7]:
import re
import pandas as pd

In [8]:
tweets_dataframe = pd.read_csv(r'/content/covid-vaccine-tweets.csv')

In [9]:
pd.set_option('max_colwidth', None)
tweets_dataframe.head()

,tweet
0,"Margorie Taylor Greene all up and down my TL. Why? Why give this nut the attention? Bigger items need addressing: Covid Vaccine, economy, impeachment, insurrection accountability."
1,"@JakeSherman @JoeBiden @SenSchumer GOP essentially let Covid run wild for a year, costing lives and money. There was no stockpile of vaccine and no plan to distribute it. GOP abandoned their responsibility to protect us. Democrats have stepped into that vacuum. Most Americans are grateful."
2,"COVID and vaccine experts, could use your help. How risky is this situation? We see our son, outside and distanced. (Have not seen him in a year) He will have received his second vaccine 4 days before. He has daily contact with COVID+ people who are not masked."
3,@Church_Militant There are no fetal cells in these COVID vaccines #FUD
4,This story started with an anonymous tip and I confirmed it. 42-year-old Napa County supervisor under fire for getting COVID-19 vaccine. https://t.co/FBW9OPD5Pz


In [10]:
def process_text(text):
  text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*','',text)
  text = re.sub(r'<.*?>', '', text)

  text = re.sub(' +', ' ', text)
  return text.strip()

In [11]:
tweets_dataframe['tweet'] = tweets_dataframe.apply(lambda row: process_text(row['tweet']), axis=1)

In [ ]:
hypothesis = "The sentiment of this tweet is {}."
candidate_labels = ['positive', 'negative']

results = classifier.predict(tweets_dataframe.tweet, candidate_labels, hypothesis=hypothesis)[0]

In [ ]:
tweets_scores_and_labels = []

for item in [sorted(list(zip(item.get('labels'), item.get('scores')))) for item in results]:
  tweets_scores_and_labels.append({a:b for a,b in item})

vaccine_hesitancy_df = pd.DataFrame(tweets_scores_and_labels)

tweets = [res.get('sequence') for res in results]

vaccine_hesitancy_df['tweets'] = tweets
vaccine_hesitancy_df = vaccine_hesitancy_df[["tweets", "positive", "negative"]]

In [ ]:
positive_tweets = vaccine_hesitancy_df[vaccine_hesitancy_df['positive'] > 0.9]

In [ ]:
positive_tweets.head(10)

In [ ]:
negative_tweets = vaccine_hesitancy_df[vaccine_hesitancy_df['positive'] > 0.9]

In [ ]:
negative_tweets.head(10)